## G - RP color Deblending 

**Input**: 

- (All) Gaia DR3 stars pre-selected for the local 50-pc sphere

- Gaia catalog of white dwarfs by Fusillo+21, GCWD21, 
with new 33 local objects as identified by Golovin+24

**Method**: Function for de-blending adopted from [A. Golovin's code](https://github.com/AlexGolovin/gaiaedr3_g_rp_synthetic.git). Also see Golovin+23 (Appendix B). 

**Output**: Input tables with a new deblended G-RP column. 


In [4]:
import numpy as np
from astropy.table import Table

from deblending import g_rp_synth

In [5]:

def apply_deblending(data, g_bp_rp, flux_cut=False):

    # Read catalog and G, G_BP, G_RP columns
    input_table = Table.read(data)

    g_column = g_bp_rp['g']
    bp_column = g_bp_rp['bp']
    rp_column = g_bp_rp['rp']

    # Create new column and fill it with the usual color 
    input_table['g_rp_deblended'] = input_table[g_column] - input_table[rp_column]

    print('Data: ',data)
    print('Total number of sources:', len(input_table['g_rp_deblended']))

    # Create mask for applying de-blending function
    # Requirements: 
    # - 0 < BP-RP < 4.25
    # - if possible, only sources with good flux-over-error (e.g. > 20)

    if flux_cut: # for Gaia source stars
        mask = np.where((input_table[bp_column] - input_table[rp_column] > 0)
                        & (input_table[bp_column] - input_table[rp_column] < 4.25)
                        & (input_table['phot_bp_mean_flux_over_error'] > 20)  # use this for MS stars!
                        & (input_table['phot_rp_mean_flux_over_error'] > 20)  
                        )[0]
    else: # for WD sample
        mask = np.where((input_table[bp_column] - input_table[rp_column] > 0)
                        & (input_table[bp_column] - input_table[rp_column] < 4.25) 
                        )[0]
    print('Sources for de-blending:', len(mask), ' - ', 
        round(len(mask)/len(input_table['g_rp_deblended'])*100,1), '%')
    
    # Sort 
    # BP-RP array needs to be in strictly increasing order!
    bp_rp_masked = np.array(input_table[bp_column] - input_table[rp_column])[mask]
    ind_sorted = bp_rp_masked.argsort()
    bp_rp_sorted = bp_rp_masked[ind_sorted]
    sorted_indices = mask[ind_sorted]

    # Calling the de-blending function:
    g_rp_deblended = g_rp_synth(bp_rp_sorted)
    print('Deblending applied')

    # Update the new color column and save
    # The new color column contains de-blended color for all sources for which the approach is aplicable, 
    # the rest of sources have a usual G-RP color. 

    input_table['g_rp_deblended'][sorted_indices] = g_rp_deblended
    filename = '.'.join(data.split('.')[:-1]) + '_deblended.' + data.split('.')[-1]
    input_table.write(filename, overwrite=True)
    print('Saved to ', filename)
    


In [6]:
# Apply to WD catalog 

data = "./data/GCWD21+Golovin24_source.csv"
g_bp_rp = {'g':'Gmag','bp':'BPmag','rp':'RPmag'}

apply_deblending(data, g_bp_rp)


Data:  ./data/GCWD21+Golovin24_source.csv
Total number of sources: 1280296
Sources for de-blending: 1134834  -  88.6 %
Deblending applied
Saved to  ./data/GCWD21+Golovin24_source_deblended.csv


In [7]:
# Apply to all Gaia stars 50-pc sample

data = "./data/GDR3_50pc_preselection_source.csv"
g_bp_rp = {'g':'phot_g_mean_mag','bp':'phot_bp_mean_mag','rp':'phot_rp_mean_mag'}

apply_deblending(data, g_bp_rp, flux_cut=True)


Data:  ./data/GDR3_50pc_preselection_source.csv
Total number of sources: 107487
Sources for de-blending: 39114  -  36.4 %
Deblending applied
Saved to  ./data/GDR3_50pc_preselection_source_deblended.csv
